# Parsing Company 10Ks From the SEC

In [1]:
# import scraping packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import os
import unicodedata
from time import time
import re

In [2]:
# import packages for getting SEC files as well as CIK numbers
# FilingType gives the 100 type of files to choose from
# Filing is the class of fetched company items
from secedgar.filings import Filing, FilingType

# CIKLookup is for looking at cik of companies, with either name or ticker, case-insensitive
from secedgar.filings.cik_lookup import CIKLookup
# get dict for all ticker/name -> CIK mapping
from secedgar.utils import get_cik_map

# datetime is needed for start_date or end_date in Filing()
from datetime import datetime


In [3]:
# get filepath 
relative_fp = os.path.realpath(os.getcwd())
fp = relative_fp + '\\industryRank_nonTop_acq.csv'
print(fp)

# load data
df = pd.read_csv(fp)
#df.drop(df.columns[[0]], axis=1, inplace=True)
tickers = df.loc[df['sic1']!=6, 'AcquirorName']#df['AcquirorName']
tickers = list(np.unique(tickers))
print(tickers)
# get CIK number for each firm
CIK_dict = {}
error_tickers = []
for ticker in tickers: 
    try: 
        CIK_dict.update(CIKLookup(ticker).lookup_dict)
    except: 
        print('error: ', ticker)
        error_tickers.append(ticker)

C:\Users\happy\OneDrive - California Institute of Technology\Desktop\Blackrock-Caltech Research\Aquitision-Database\code\industryRank_nonTop_acq.csv
['AECOM Technology Corp', 'Airgas Inc', 'Allergan Inc', 'Allergan PLC', 'Bed Bath & Beyond Inc', 'BioMarin Pharmaceutical Inc', 'Bottomline Technologies Inc', 'CAVU Resources Inc', 'Carver Corp', 'Comcast Corp', 'DaVita HealthCare Partners Inc', 'DaVita Inc', 'Delek Us Holdings Inc', 'Dun & Bradstreet Corp', 'Emerson Electric Co', 'Federal Signal Corp', 'Ferro Corp', 'Hanger Inc', 'Hanger Orthopedic Group Inc', 'Harman Intl Industries Inc', 'Hercules Offshore Inc', 'Latteno Food Corp', 'Measurement Specialties Inc', 'Microsoft Corp', 'Mistras Group Inc', 'Monotype Imaging Holdings Inc', 'NCI Inc', 'NuStar Energy LP', 'Omnicare Inc', 'Papa Bello Enterprises Inc', 'Regency Energy Partners LP', 'ResMed Inc', 'Superior Well Services Inc', 'Time Warner Cable Inc', 'VCA Antech Inc', 'VCA Inc', 'Vanguard Natural Resources LLC', 'Vonage Holdings C

C:\Users\happy\Anaconda3\lib\site-packages\secedgar\filings\cik_validator.py:113: UserWarning: Lookup 'Airgas Inc' will be skipped.
                          Found multiple companies matching 'Airgas Inc':
                          AIRGAS INCSIC: 5084 - WHOLESALE-INDUSTRIAL MACHINERY & EQUIPMENT
AIRGAS INC EMPLOYEE BENEFITS TRUST
  warnings.warn(warning_message)


error:  CAVU Resources Inc


C:\Users\happy\Anaconda3\lib\site-packages\secedgar\filings\cik_validator.py:113: UserWarning: Lookup 'Comcast Corp' will be skipped.
                          Found multiple companies matching 'Comcast Corp':
                          COMCAST HOLDINGS CORPSIC: 4841 - CABLE & OTHER PAY TELEVISION SERVICES
COMCAST CORPSIC: 4841 - CABLE & OTHER PAY TELEVISION SERVICES
  warnings.warn(warning_message)


error:  Delek Us Holdings Inc


C:\Users\happy\Anaconda3\lib\site-packages\secedgar\filings\cik_validator.py:113: UserWarning: Lookup 'Dun & Bradstreet Corp' will be skipped.
                          Found multiple companies matching 'Dun & Bradstreet Corp':
                          DEX ONE CorpSIC: 7310 - SERVICES-ADVERTISING
MOODYS CORP /DE/SIC: 7320 - SERVICES-CONSUMER CREDIT REPORTING, COLLECTION AGENCIES
DUN & BRADSTREET CORP/NWSIC: 7320 - SERVICES-CONSUMER CREDIT REPORTING, COLLECTION AGENCIES
  warnings.warn(warning_message)


error:  Hanger Inc
error:  Harman Intl Industries Inc
error:  Hercules Offshore Inc
error:  Mistras Group Inc
error:  NCI Inc
error:  NuStar Energy LP
error:  Superior Well Services Inc
error:  Vanguard Natural Resources LLC
error:  Zayo Group Holdings Inc


In [4]:
print(len(CIK_dict), ' CIK Available')
print(len(tickers), ' Tickers Given')
#print(error_tickers)
print([i for i in tickers if i not in list(CIK_dict.keys())])

26  CIK Available
40  Tickers Given
['Airgas Inc', 'CAVU Resources Inc', 'Comcast Corp', 'Delek Us Holdings Inc', 'Dun & Bradstreet Corp', 'Hanger Inc', 'Harman Intl Industries Inc', 'Hercules Offshore Inc', 'Mistras Group Inc', 'NCI Inc', 'NuStar Energy LP', 'Superior Well Services Inc', 'Vanguard Natural Resources LLC', 'Zayo Group Holdings Inc']


In [5]:
'''
from secedgar.utils import get_cik_map

a= get_cik_map(key="title")
for ticker in tickers:
    if ticker.upper() not in a.keys():
        print(ticker)
'''
missing = {
    'Airgas Inc': '0000804212', 
    'CAVU Resources Inc': '0001461679', 
    'Comcast Corp': '0001166691', 
    'Delek Us Holdings Inc': '0001694426',
    'Dun & Bradstreet Corp': '0001115222',
    'Hanger Inc': '0000722723', 
    'Harman Intl Industries Inc': '0000800459', 
    'Hercules Offshore Inc': '0001330849', 
    'Mistras Group Inc': '0001436126', 
    'NCI Inc': '0001334478', 
    'NuStar Energy LP': '0001110805', 
    'Superior Well Services Inc': '0001323715', 
    'Zayo Group Holdings Inc': '0001608249'
}
#'American Campus Comm Inc': '0001283630', this is bank/insurance/real estate

fucked = ['Vanguard Natural Resources LLC']
CIK_dict.update(missing)
CIK_dict

{'AECOM Technology Corp': '0000868857',
 'Allergan Inc': '0000850693',
 'Allergan PLC': '0001578845',
 'Bed Bath & Beyond Inc': '0000886158',
 'BioMarin Pharmaceutical Inc': '0001048477',
 'Bottomline Technologies Inc': '0001073349',
 'Carver Corp': '0000766177',
 'DaVita HealthCare Partners Inc': '0000927066',
 'DaVita Inc': '0000927066',
 'Emerson Electric Co': '0000032604',
 'Federal Signal Corp': '0000277509',
 'Ferro Corp': '0000035214',
 'Hanger Orthopedic Group Inc': '0000722723',
 'Latteno Food Corp': '0000743241',
 'Measurement Specialties Inc': '0000778734',
 'Microsoft Corp': '0000789019',
 'Monotype Imaging Holdings Inc': '0001385292',
 'Omnicare Inc': '0000353230',
 'Papa Bello Enterprises Inc': '0001461145',
 'Regency Energy Partners LP': '0001338613',
 'ResMed Inc': '0000943819',
 'Time Warner Cable Inc': '0001377013',
 'VCA Antech Inc': '0000817366',
 'VCA Inc': '0000817366',
 'Vonage Holdings Corp': '0001272830',
 'WESCO International Inc': '0000929008',
 'Airgas Inc':

In [6]:
# store the info for each firm in df, before we start web scraping 10Ks
firms_info = []

for company_ticker in tickers:
    if company_ticker not in CIK_dict.keys():
        continue
    company_cik = CIK_dict[company_ticker]
    
    # base url for every search 10-K page on SEC.org
    search_10K_page_base_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=|&type=10-k'

    # get firm-specific url
    search_10K_page_url = search_10K_page_base_url.replace('|', company_cik)

    # connect to page and search
    search_page = requests.get(search_10K_page_url).content
    soup = BeautifulSoup(search_page, 'html')

    # store report numbers for each firm
    ###can do later

    # find the section denoting report number
    for td_instance in soup.find_all('td',{'class':'small'}):
        text = td_instance.text
        
        # skip 10-K/A files which are amendments
        if '[Amend]' in text:
            #print('10K/A file encountered')
            continue
        
        # parse to get each report number
        index_of_start = text.find('Acc-no:')
        index_of_end = text.find('(34 Act)') # stop before '('

        # some old ones has no (34 Act)
        if index_of_end == -1:
            index_of_end = text.find('Size:')

        if index_of_start == -1 or index_of_end == -1:
            # guess no annual report in here
            print('no annual report in this <td> class')
            print(td_instance.text)
            continue

        index_of_start += len('Acc-no:') # start after this string

        # get report number
        report_number = text[index_of_start: index_of_end]
        # remove - and whitespace
        report_number = report_number.strip().replace('-', '')
        # find date
        report_date = td_instance.findNext('td').text
        # update df
        firms_info.append([company_ticker, company_cik, report_number, report_date])

#convert to dataframe
firms_info = pd.DataFrame(firms_info, columns=['Ticker', 'CIK', '10-K Report Number', '10-K Report Date'])

In [7]:
firms_info.head()

,Ticker,CIK,10-K Report Number,10-K Report Date
0,AECOM Technology Corp,0000868857,000110465919063367,2019-11-13
1,AECOM Technology Corp,0000868857,000104746918007225,2018-11-13
2,AECOM Technology Corp,0000868857,000104746917007051,2017-11-14
3,AECOM Technology Corp,0000868857,000104746916016739,2016-11-16
4,AECOM Technology Corp,0000868857,000104746915008965,2015-11-25


In [8]:
firms_info.shape

(636, 4)

In [9]:
#get_10_K_page_url
def get_10_K_page_url(xml_summary):
    page_10K_url = xml_summary.replace('/FilingSummary.xml', '')
    page_10K_url = page_10K_url[:-8] + '-' + page_10K_url[-8:]
    page_10K_url = page_10K_url[:-6] + '-' + page_10K_url[-6:]
    page_10K_url += '-index.htm'
    return page_10K_url

#get 10 K html text file
def get_10_K_txt(xml_summary):
    page_10K_url = get_10_K_page_url(xml_summary)
    return page_10K_url.replace('-index.htm', '.txt')
get_10_K_page_url('https://www.sec.gov/Archives/edgar/data/0000892482/000089248220000007/FilingSummary.xml')
#get_10_K_txt('https://www.sec.gov/Archives/edgar/data/0000892482/000089248220000007/FilingSummary.xml')

'https://www.sec.gov/Archives/edgar/data/0000892482/0000892482-20-000007-index.htm'

In [10]:
# decode weird strings...not really working
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

In [11]:
############################################
# helper functions for reports without .xlm summary
# older ones before 2012
############################################
def parse_acquisition_table(table_instance): # a bs4 table instance
    acquisition_related = False
    
    # define a dictionary that will store the different parts of the statement.
    table_data = {}
    table_data['headers'] = []
    table_data['sections'] = []
    table_data['data'] = []

    # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
    for index, row in enumerate(table_instance.find_all('tr')):
        
        # first let's get all the elements.
        cols = row.find_all('td')
        
        # get data by row
        reg_row = [ele.text.strip() for ele in cols] # remove unicode char
        table_data['data'].append(reg_row)
        
        keywords = ['acquisitions', 'business combinations'] #!!!!!!
        if True in [keyword in element.lower() for element in reg_row for keyword in keywords]:
            acquisition_related = True
        '''
        # if it's a regular row and not a section or a table header
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
            reg_row = [ele.text.strip() for ele in cols]
            table_data['data'].append(reg_row)
                        
            if True in ['acquisition' in element.lower() for element in reg_row]:
                acquisition_related = True
            
        # if it's a regular row and a section but not a table header
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            table_data['sections'].append(sec_row)
            
            if True in ['acquisition' in element.lower() for element in sec_row]:
                acquisition_related = True
            
        # finally if it's not any of those it must be a header
        elif (len(row.find_all('th')) != 0):            
            hed_row = [unicodedata.normalize('NFKD',ele.text.strip()) for ele in row.find_all('th')]
            table_data['headers'].append(hed_row)
             
            if True in ['acquisition' in element.lower() for element in hed_row]:
                acquisition_related = True
            
        else:            
            print('We encountered an error.')
        ''' 
    '''
    # turn to dataframe
    table = pd.DataFrame(table_data['data'])
    if acquisition_related: 
        # only return when it is acquisition related
        return table
    return # return None if not
    '''
    return acquisition_related
'''
# get acquisition related reports for older 10-Ks using .txt html url, return a list of urls where we save them
def get_info_from_txt(txt_url, name):
    content = requests.get(txt_url).content
    soup = BeautifulSoup(content, 'html')
    
    # get notes sections for acquisition
    stuff = []
    bolds = soup.find_all('b')
    if bolds is not None:
        for bold_instance in bolds:
            #bold_font = bold_instance.findChild('font' , recursive=False)
            if bold_instance is not None:
                note_html = str(bold_instance)
                bold_text = bold_instance.text
                
                keywords = ['acquisitions', 'business combinations'] #!!!!!!
                if True in [keyword in bold_text.lower() for keyword in keywords]:
                    #first check if it is actually a title
                    # 60 is a like loose upper bound for a note title pretty sure
                    if len(' '.join(bold_text.split())) > 60: 
                        continue                    
                    
                    # just parse next 4 pages the most stupid way 
                    # but it ensures I can get everything
                    pages = 1
                    print(bold_text)
                    thematic_break = bold_instance.find_next('hr',{'width':'100%'}) # page break
                    if thematic_break is None: 
                        #try again by just finding hr
                        thematic_break = bold_instance.find_next('hr')
                    if thematic_break is None: 
                        # reached end of 10K
                        continue
                    current_instance = bold_instance.find_next()
                    while pages <= 4 and current_instance is not None:
                        note_html += str(current_instance)
                        if current_instance == thematic_break: 
                            pages += 1    
                        current_instance = current_instance.find_next()
                    note_html += str(thematic_break)
                    stuff.append(note_html)
                    #############################
                    # this is the next note section's title
                    next_bold_instance = bold_instance.find_next('b')
                    instance = next_bold_instance
                    while instance != next_bold_instance:
                        # new way
                        note_html += str(instance)
                        instance = instance.find_next()
                    div_to_stop = next_bold_instance.parent #maybe find parent div
                    # the paragraphs we want
                    div_instance = bold_instance.find_next('div')
                    while div_instance != div_to_stop:
                        #paragraph = unicodedata.normalize('NFKD', div_instance.text)
                        #acquisition_notes += paragraph
                        # new way
                        note_html += str(div_instance)
                        div_instance = div_instance.find_next('div')
                    #stuff.append(acquisition_notes.encode('ascii', 'ignore')) # remove unicode char
                    # new way
                    ##############################   
    
    # get table sections for acquisition
    tables = soup.find_all('table')
    if tables is not None:
        for table_instance in tables:
                acquisition_related = parse_acquisition_table(table_instance)
                if acquisition_related: # if table is not None:
                    stuff.append(table_instance)
    
    report_urls = []
    # for all related stuff, write to html and get the link for them
    if stuff:
        for index, html_instance in enumerate(stuff):
            report_url = add_link_and_write(html_instance, name + '_report_' + str(index), dir_name='sec_data')
            report_urls.append(['report ' + str(index), report_url])
    return report_urls
'''
# add a parent tag for the html instanse parsed
def wrap(to_wrap, wrap_in):
    contents = to_wrap.replace_with(wrap_in)
    wrap_in.append(contents)
    
# add link to the html instance we have s.t. we can write it to a html file and call later
def add_link_and_write(html_instance, link_name, dir_name='sec_data'):
    # turn tag into a bs4 instance
    #info = BeautifulSoup(str(html_instance))
    
    # why I need this????cannot recall
    #wrap(info.table, info.new_tag('a'))
    #url = r"file:///C:\Users\happy\OneDrive - California Institute of Technology\Desktop\{}\{}.html".format(dir_name, link_name)
    #info.a['href'] = url
    #<a href="where/you/want/the/link/to/go">text of the link</a>
    
    # write to html
    dir_name = dir_name.replace(' ', '_')
    link_name = link_name.replace(' ', '_')
    f = open(os.path.realpath(os.getcwd())+'\\{}\\{}.html'.format(dir_name, link_name),'wb')
    f.write(html_instance.encode('ascii', 'ignore'))
    #f.write(str(info).encode('ascii', 'ignore'))
    f.close()
    return r'http://makeresearchgreatagain.com/sec_files/' + link_name + '.html'

In [12]:
# this becomes a helper function
def get_info_from_txt_single_instance(txt_url, year, name='', max_word=50, prev_year_tolerance=10, max_pages=5, instance_type='b'):
    content = requests.get(txt_url).content
    soup = BeautifulSoup(content, 'html')
    
    # get notes sections for acquisition
    stuff = []
    bolds = soup.find_all(instance_type)
    
    # special case designed for microsoft since its section title is underlined, not bold
    if '/0000789019/' in txt_url: # 0000789019 is MSFT CIK
        bolds = soup.find_all('u')
        
    if bolds is not None:
        #change to while loop to skip the bold instances already seen in previous parsing
        i = 0 # index
        while i < len(bolds):        
            bold_instance = bolds[i]
            #bold_font = bold_instance.findChild('font' , recursive=False)
            if bold_instance is not None:
                bold_text = bold_instance.text
                
                note_titles = ['acquisition', 'business combination'] #!!!!!
                # special case
                if '/0001377013/' in txt_url: # 0000789019 is Time Warner Cable CIK
                    note_titles = ['acquisition', 'business combination', 'transaction']
                    
                if True in [title in bold_text.lower() for title in note_titles]:
                    #first check if it is actually a title
                    # 60 is a like loose upper bound for a note title pretty sure
                    if len(' '.join(bold_text.split())) > 60:
                        i += 1 #increment
                        continue
                    
                    #note_html = ''
                    
                    #case 1: note N is present in title: 
                    # easy...keep going until hit the bold instance with note N + 1
                    '''
                    not working
                    if 'note' in bold_text.lower():
                        print('case 1')
                        title_words_list = bold_text.lower().split()
                        index_of_note = title_words_list.index('note')
                        if not title_words_list[index_of_note+1].isnumeric():
                            raise Exception('this note section has no number!!!!!')
                        else: 
                            next_note_header = 'note' + str(int(title_words_list[index_of_note+1]) + 1)
                            print(next_note_header)
                            note_html = str(bold_instance)
                            current_instance = bold_instance.find_next()
                            while current_instance is not None and next_note_header not in current_instance.text.lower().replace(' ', ''):
                                note_html += str(current_instance)   
                                current_instance = current_instance.find_next()
                    '''
                    #case 2: use this 5 pages info as worst case html
                    # just parse next 5 pages the most stupid way 
                    # but it ensures I can get everything
                    #else: 
                    pages = 1
                    note_html = str(bold_instance)
                    thematic_break = bold_instance.find_next('hr',{'width':'100%'}) # page break
                    if thematic_break is None: 
                        #try again by just finding hr
                        thematic_break = bold_instance.find_next('hr')
                    if thematic_break is None: 
                        # reached end of 10K
                        i += 1 #increment
                        continue
                    current_instance = bold_instance.find_next()
                    while pages <= max_pages and current_instance is not None:
                        note_html += str(current_instance)
                        #print(current_instance.name)
                        if current_instance == thematic_break:#'<hr' in str(current_instance): 
                            pages += 1
                        current_instance = current_instance.find_next()
                    note_html += str(thematic_break)
                    
                    print('finished get html')
                    
                    #now we have the html we want to refine on
                    #get its text
                    #return note_html
                    note_text = BeautifulSoup(note_html.encode('ascii', 'ignore').decode().replace('\n', ' ')).text
                    note_text = note_text.split()
                    
                    #after passing max_char wihout seeing a keyword, we stop and get length of the strings so far
                    prev_year_tolerance = prev_year_tolerance #give a tolerance for seeing year other than X and X-1
                    prev_year_encountered = 0
                    distance = 0
                    curr_index = 0
                    #also need to count for $ since we might encounter a table that is relevant
                    keywords = ['acquisition', 'acquire', 'merger', 'business combination', 'asset', 'million', 
                                'billion', 'thousand', 'fair', 'value', 'purchase', 'price', 'paid', '$', 
                                'agreement', 'transaction', 'ownership', 'bought', 'cost'] #, 'goodwill'
                    previous_19_years = np.arange(int(year)-20, int(year)-1).astype(str) #previous fiscal years except last fiscal year in string
                    
                    while distance < max_word:
                        if curr_index >= len(note_text) or prev_year_encountered > prev_year_tolerance: 
                            break
                        if True in [prev_year in note_text[curr_index] for prev_year in previous_19_years]:
                            prev_year_encountered += 1 # do not want this year's report to report acquisitions in previous years
                        if True in [keyword in note_text[curr_index].lower() for keyword in keywords]:
                            distance = 0
                        else: 
                            distance += 1 #len(note_text[curr_index])
                        curr_index += 1
                    
                    refined_text = note_text[:curr_index-1]
                    
                    print('finished get text')
                    '''
                    #compute information ratio = length(refined_text) / length(note_text)
                    info_ratio = len(' '.join(refined_text)) / len(' '.join(note_text))
                    
                    #find the length we want for html instance
                    desired_length = round(len(note_html) * info_ratio)
                    
                    refined_html = note_html[:desired_length]
                    refined_html = refined_html[:(refined_html.rfind('>')+1)]
                    
                    print(refined_html[-100:])
                    '''
                    desired_length = len(''.join(refined_text))
                    refined_html = str(bold_instance)
                    current_instance = bold_instance
                    prev_instance = ''
                    dummy_html = str(bold_instance)
                    #used another way: re to parse only text
                    while len(re.sub("\<.*?\>", "", refined_html).replace(' ', '').replace('\n', '')) < desired_length: 
                        if len(dummy_html) > len(note_html) or current_instance is None:
                            break
                        prev_instance = current_instance
                        current_instance = current_instance.find_next()
                        
                        #checks if another bold instance appeared
                        if '<b>' in str(current_instance) and True in [title in current_instance.text.lower() for title in note_titles]:
                            i += 1 #skip next instance
                        
                        while re.sub("\<.*?\>", "", str(prev_instance)) == re.sub("\<.*?\>", "", str(current_instance)) or prev_instance.text.lower() == current_instance.text.lower():
                            prev_instance = current_instance
                            current_instance = current_instance.find_next()
                            dummy_html += str(current_instance)
                            if '<b>' in str(current_instance) and True in [title in current_instance.text.lower() for title in note_titles]:
                                i += 1 #skip next instance
                                
                        if str(prev_instance) == str(current_instance):
                            print('previous and current instance are the same')
                            
                        refined_html += str(current_instance)
                        dummy_html += str(current_instance)
                    
                    print('finished refine html')
            
                    #update
                    stuff.append(refined_html)
            i += 1 #increment i
    '''
    skip cuz we want to write aggregately to one file per note
    report_urls = []
    # for all related stuff, write to html and get the link for them
    if stuff:
        for index, html_instance in enumerate(stuff):
            report_url = add_link_and_write(html_instance, name + '_report_' + str(index), dir_name='sec_data')
            report_urls.append(['report ' + str(index), report_url])
    return report_urls
    '''
    return stuff, len(stuff)


In [38]:
# get acquisition related reports for older 10-Ks using .txt html url, return a list of urls where we save them
def get_info_from_txt(txt_url, year, name='', max_word=50, prev_year_tolerance=10, max_pages=5):
    html = ''
    N = 0
    instances = ['b', 'h1', 'strong', 'div'] #'p' and 'td, 'u'should be checked but too common
    for instance in instances:
        print('instance:', instance)
        curr_html, curr_N = get_info_from_txt_single_instance(txt_url, year, name, max_word, prev_year_tolerance, 
                                                      max_pages, instance_type=instance)
        if curr_N > 0:
            html = curr_html # a list
            N = curr_N
            return html, N
    return html, N

In [39]:
link =r'https://www.sec.gov/Archives/edgar/data/0001166691/0001193125-09-033975.txt'
#link = r'https://www.sec.gov/Archives/edgar/data/0000789019/0001193125-11-200680.txt'
instance = get_info_from_txt(link, '11111', '2009')

instance: b
finished get html
finished get text
finished refine html


In [15]:
# gets the end date of the fiscal year 
def get_fiscal_year(page_10_K_url):
    content = requests.get(page_10_K_url).content
    soup = BeautifulSoup(content, 'html')
    
    # get head titles in 10K page sections for acquisition
    head_titles = soup.find_all('div', {'class':'infoHead'})
    
    date_ended = None
    for header in head_titles:
        if header.text.lower() == 'period of report':
            date_ended = header.find_next('div', {'class':'info'}).text
    return date_ended

In [16]:
xxxxx = 'https://www.sec.gov/Archives/edgar/data/0001115222/0001115222-16-000045-index.htm'
get_fiscal_year(xxxxx)

'2015-12-31'

In [17]:
def get_table(txt_url, name=''):
    content = requests.get(txt_url).content
    soup = BeautifulSoup(content, 'html')
    
    # get notes sections for acquisition
    tables = soup.find_all('table')
    print(len(tables))
    
    tables_url = []
    if tables is not None:
        for i in range(len(tables)):
            tables_url.append(add_link_and_write(str(tables[i]), name + '_' + str(i), dir_name='table'))
    return tables_url

In [18]:
link =r'https://www.sec.gov/Archives/edgar/data/0001166691/0001193125-09-033975.txt'
get_table = get_table(link, 'table')

497


In [19]:
BeautifulSoup(str('<hr align="CENTER" noshade="" size="4" width="100%"/>')).name

'[document]'

In [ ]:
link =r'https://www.sec.gov/Archives/edgar/data/0001166691/0001193125-09-033975.txt'
#instance = get_info_from_txt(link, '11111', '2009')

In [20]:
'<hr align="CENTER" noshade="" size="4" width="100%"/>'.index('<hr')

0

In [21]:
'''
1. used info ratio to estimate the length of html to get
2. upper limit is 4 pages
3. immediately stop adding html if fiscal year 20XX earlier than last fiscal year shows up
4. for .xlm omits tables with name detail or table on it...only want general note
'''

'\n1. used info ratio to estimate the length of html to get\n2. upper limit is 4 pages\n3. immediately stop adding html if fiscal year 20XX earlier than last fiscal year shows up\n4. for .xlm omits tables with name detail or table on it...only want general note\n'

In [22]:
#print(str(instance).lower().index('</'))
#print(str(instance).lower().index('acquisitions'))
#len(str(instance)[6:184])
#len('note 2 . business combinations')

In [23]:
# main function
def find_relevent_reports(firms_info, start_year=2007, end_year=2017, max_word=50, 
                          prev_year_tolerance=10, max_pages=5):
    # to be returned
    relevant_table_links = []
    # to be returned and saved for Professor
    #relevant_table_links = pd.DataFrame(' ', index=list(firms_info['Ticker'].unique()), columns=np.arange(1950,2021)[::-1].astype(str))
    
    # failed to parse one
    failures = 0
    failed_table_links = []
    
    # base link for every xlm doc for each 10-K
    xlm_base_url = r'https://www.sec.gov/Archives/edgar/data/CIK/report_number/FilingSummary.xml'
    
    # for each 10-K we have
    for i in range(firms_info.shape[0]):
        
        #if i < 30 or i > 40:# or i > 200:
        #    continue
        #if report_info_df['Ticker'] != 'ATVI':
        #    continue
        print(i)
        
        # current row
        report_info_df = firms_info.iloc[i,:]
        current_year = int(report_info_df['10-K Report Date'].split('-')[0])
        
        #if report_info_df['Ticker'] != 'Dun & Bradstreet Corp':# or current_year != 2010:
        #    continue

        if current_year > int(end_year) or current_year < int(start_year):
            continue
            
        print(report_info_df['Ticker'])
        print(report_info_df['10-K Report Date'])
        print('-'*100)
        
        # xlm link for current 10-K
        xml_summary = xlm_base_url.replace('CIK', report_info_df['CIK']).replace('report_number', report_info_df['10-K Report Number'])
    
        # define a new base url that represents the filing folder. This will come in handy when we need to download the reports.
        base_url = xml_summary.replace('FilingSummary.xml', '')
        
        print(xml_summary)
        print(get_10_K_page_url(xml_summary))
        print(get_10_K_txt(xml_summary))
        
        try:
            # request and parse the content
            content = requests.get(xml_summary).content
            soup = BeautifulSoup(content, 'lxml')
            
        except:
            #print(report_info_df['Company Name'])
            print('Link does not exist')
            print(xml_summary)
            print('-'*100)

        # find the 'myreports' tag because this contains all the individual reports submitted.
        reports = soup.find('myreports')
        page_10_K_url = get_10_K_page_url(xml_summary)
        txt_url = get_10_K_txt(xml_summary)
        end_date_fiscal_year = get_fiscal_year(page_10_K_url)
        
        #content to write altogether for each company-year if not .xlm
        report_html = ''
        #num_reports = 0
        
        if reports == None:
            # this is the case when the 10-Ks are early than (including 2011)
            # no .xlm summary exist so directly parse .txt html file
            print('Directly parse .txt html file')
            try:              
                html_contents, N = get_info_from_txt(txt_url, year=current_year, max_word=max_word, 
                                                     prev_year_tolerance=prev_year_tolerance, max_pages=max_pages)
                report_html += '<br /><hr noshade=""/><hr noshade=""/><br />'.join(html_contents)
                #num_reports += N
                
                '''
                reports_url = get_info_from_txt(txt_url, name=report_info_df['Ticker']+'_'+report_info_df['10-K Report Date'], 
                                                year=current_year, max_word=max_word)
                
                # of form [report name, report url]
                for report_name_and_url in reports_url:

                    relevant_table_links.append(report_info_df.tolist() + [report_name_and_url[1], page_10_K_url])
                    #relevant_table_links.loc[report_info_df['Ticker'], report_info_df['10-K Report Date'].split('-')[0]] += report_name_and_url[1] + ' | '

                    print(page_10_K_url)
                    print(report_name_and_url[0])
                    print(report_name_and_url[1])
                    print('-'*100)
                '''

            # real issue didnt catch
            except Exception as e:
                print(e)
                print('got fucked 1!')
                print(txt_url)
                print('-'*100)
                failures += 1
                failed_table_links.append([report_info_df['Ticker'], report_info_df['10-K Report Date'], page_10_K_url, txt_url])
                continue
            
        else: 
            # loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
            try:
                count = 0 
                for report in reports.find_all('report')[:-1]:

                    # acquisition related
                    # what we want
                    if 'acquisition' in report.shortname.text.lower() or 'aquisition' in report.longname.text.lower():
                        
                        #####we dont want tables and detailed stuff for RA, only note
                        forms_not_desired = ['table', 'detail']
                        if True in [form in report.shortname.text.lower() for form in forms_not_desired]:
                            continue

                        # update our list
                        table_name = report.shortname.text # report.longname.text # both long or short is fine
                        table_url = base_url + report.htmlfilename.text
                        
                        relevant_table_links.append(report_info_df.tolist() + [table_url, page_10_K_url, end_date_fiscal_year])
                        #relevant_table_links.append(report_info_df.tolist() + [table_url, page_10_K_url, ', '.join([str(current_year), str(current_year-1)])])
                        #relevant_table_links.loc[report_info_df['Ticker'], report_info_df['10-K Report Date'].split('-')[0]] += table_url + ' | '
                        count += 1
                        print(page_10_K_url)
                        print(table_name)
                        print(table_url)
                        print('-'*100)
                if count == 0:
                    relevant_table_links.append(report_info_df.tolist() + ['No Acquisition', page_10_K_url, end_date_fiscal_year])
                
                report_html = 'got xlm' # for blank row printing
                        
            except:
                # some .xlm summary have acquisition related report but not .htm instance
                # need parse .html directly as well
                print('Directly parse .txt html file')
                try:
                    html_contents, N = get_info_from_txt(txt_url, year=current_year, max_word=max_word, 
                                                     prev_year_tolerance=prev_year_tolerance, max_pages=max_pages)
                    report_html += '<br /><hr noshade=""/><hr noshade=""/><br />'.join(html_contents)
                    #num_reports += N
                    
                    '''
                    reports_url = get_info_from_txt(txt_url, name=report_info_df['Ticker']+'_'+report_info_df['10-K Report Date'], 
                                                    year=current_year, max_word=max_word)

                    # of form [report name, report url]
                    for report_name_and_url in reports_url:
                        
                        relevant_table_links.append(report_info_df.tolist() + [report_name_and_url[1], page_10_K_url])
                        #relevant_table_links.loc[report_info_df['Ticker'], report_info_df['10-K Report Date'].split('-')[0]] += report_name_and_url[1] + ' | '
                        
                        print(page_10_K_url)
                        print(report_name_and_url[0])
                        print(report_name_and_url[1])
                        print('-'*100)
                    '''

                # real issue didnt catch
                except Exception as e:
                    print(e)
                    print('got fucked 2!')
                    print(txt_url)
                    failures += 1
                    failed_table_links.append([report_info_df['Ticker'], report_info_df['10-K Report Date'], page_10_K_url, txt_url])
                    continue
        
        #write html content to file if not none
        if report_html.strip() != '':
            if report_html != 'got xlm':
                report_url = add_link_and_write(report_html, report_info_df['Ticker']+'_'+report_info_df['10-K Report Date'] + '_report', dir_name='sec_data')
                relevant_table_links.append(report_info_df.tolist() + [report_url, page_10_K_url, end_date_fiscal_year])
                print(report_url)
        else:
            relevant_table_links.append(report_info_df.tolist() + ['No Acquisition', page_10_K_url, end_date_fiscal_year])
                                                
                
                
    relevant_table_links = pd.DataFrame(relevant_table_links, columns=['Compnay Name', 'CIK', '10-K Report Number', '10-K Report Date',
                                                                       'Notes Link', '10-K Report Page URL', 'End Date of Fiscal Year'])
    relevant_table_links = relevant_table_links.drop(columns=['10-K Report Number', '10-K Report Date'])
    
    failed_table_links = pd.DataFrame(failed_table_links, columns=['Compnay Name', '10-K Report Date', '10-K Report Page URL', '10-K Report .html URL'])

    return relevant_table_links, failed_table_links, failures

In [40]:
start = time()

In [41]:
relevant_table_links, failed_tables, failed_count = find_relevent_reports(firms_info, start_year=2007, 
                                                                          end_year=2017, max_word=70, 
                                                                          prev_year_tolerance=10, max_pages=5)

0
1
2
AECOM Technology Corp
2017-11-14
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000868857/000104746917007051/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000868857/0001047469-17-007051-index.htm
https://www.sec.gov/Archives/edgar/data/0000868857/0001047469-17-007051.txt
https://www.sec.gov/Archives/edgar/data/0000868857/0001047469-17-007051-index.htm
Business Acquisitions, Goodwill and Intangible Assets
https://www.sec.gov/Archives/edgar/data/0000868857/000104746917007051/R10.htm
----------------------------------------------------------------------------------------------------
3
AECOM Technology Corp
2016-11-16
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000868857/000104746916016739/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000868857/0001047469-16-016739-ind

https://www.sec.gov/Archives/edgar/data/0000804212/0000804212-16-000048-index.htm
Acquisitions
https://www.sec.gov/Archives/edgar/data/0000804212/000080421216000048/R12.htm
----------------------------------------------------------------------------------------------------
14
Airgas Inc
2015-05-27
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000804212/000080421215000014/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000804212/0000804212-15-000014-index.htm
https://www.sec.gov/Archives/edgar/data/0000804212/0000804212-15-000014.txt
https://www.sec.gov/Archives/edgar/data/0000804212/0000804212-15-000014-index.htm
Acquisitions and Divestitures
https://www.sec.gov/Archives/edgar/data/0000804212/000080421215000014/R11.htm
----------------------------------------------------------------------------------------------------
15
Airgas Inc
2014-05-29
--------------------------------------

finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get 

63
Allergan PLC
2014-02-25
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0001578845/000119312514066242/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0001578845/0001193125-14-066242-index.htm
https://www.sec.gov/Archives/edgar/data/0001578845/0001193125-14-066242.txt
https://www.sec.gov/Archives/edgar/data/0001578845/0001193125-14-066242-index.htm
Acquisitions and Other Agreements
https://www.sec.gov/Archives/edgar/data/0001578845/000119312514066242/R11.htm
----------------------------------------------------------------------------------------------------
64
65
66
67
Bed Bath & Beyond Inc
2017-04-25
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000886158/000117184317002300/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000886158/0001171843-17-002300-index.htm
https://www.sec

https://www.sec.gov/Archives/edgar/data/0001048477/0001564590-15-001140-index.htm
ACQUISITION OF SAN RAFAEL CORPORATE CENTER
https://www.sec.gov/Archives/edgar/data/0001048477/000156459015001140/R15.htm
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0001048477/0001564590-15-001140-index.htm
ACQUISITION OF ZACHARON PHARMACEUTICALS, INC.
https://www.sec.gov/Archives/edgar/data/0001048477/000156459015001140/R29.htm
----------------------------------------------------------------------------------------------------
95
BioMarin Pharmaceutical Inc
2014-02-26
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0001048477/000119312514070025/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0001048477/0001193125-14-070025-index.htm
https://www.sec.gov/Archives/edgar/data/0001048477/0001193125-14-070025.txt


...

finished refine html
http://makeresearchgreatagain.com/sec_files/Superior_Well_Services_Inc_2009-03-13_report.html
542
Superior Well Services Inc
2008-03-11
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0001323715/000095015208001841/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0001323715/0000950152-08-001841-index.htm
https://www.sec.gov/Archives/edgar/data/0001323715/0000950152-08-001841.txt
Directly parse .txt html file
instance: b
finished get html
finished get text
finished refine html
http://makeresearchgreatagain.com/sec_files/Superior_Well_Services_Inc_2008-03-11_report.html
543
Superior Well Services Inc
2007-03-09
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0001323715/000095015207001954/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0001323715/0000950152-07-001954-

finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
http://makeresearchgreatagain.com/sec_files/Time_Warner_Cable_Inc_2007-02-23_report.html
555
VCA Antech Inc
2017-02-28
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000817366/000162828017001885/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000817366/0001628280-17-001885-index.htm
https://www.sec.gov/Archives/edgar/data/0000817366/0001628280-17-001885.txt
https://www.sec.gov/A

https://www.sec.gov/Archives/edgar/data/0000817366/0001628280-17-001885-index.htm
Acquisitions
https://www.sec.gov/Archives/edgar/data/0000817366/000162828017001885/R13.htm
----------------------------------------------------------------------------------------------------
576
VCA Inc
2016-02-26
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000817366/000162828016011807/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000817366/0001628280-16-011807-index.htm
https://www.sec.gov/Archives/edgar/data/0000817366/0001628280-16-011807.txt
https://www.sec.gov/Archives/edgar/data/0000817366/0001628280-16-011807-index.htm
Acquisitions
https://www.sec.gov/Archives/edgar/data/0000817366/000162828016011807/R13.htm
----------------------------------------------------------------------------------------------------
577
VCA Inc
2015-02-27
-----------------------------------------------------------

https://www.sec.gov/Archives/edgar/data/0001272830/0001272830-15-000025-index.htm
Acquisition of Business
https://www.sec.gov/Archives/edgar/data/0001272830/000127283015000025/R19.htm
----------------------------------------------------------------------------------------------------
601
Vonage Holdings Corp
2014-02-13
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0001272830/000127283014000020/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0001272830/0001272830-14-000020-index.htm
https://www.sec.gov/Archives/edgar/data/0001272830/0001272830-14-000020.txt
https://www.sec.gov/Archives/edgar/data/0001272830/0001272830-14-000020-index.htm
Acquisition of Business
https://www.sec.gov/Archives/edgar/data/0001272830/000127283014000020/R19.htm
----------------------------------------------------------------------------------------------------
602
Vonage Holdings Corp
2013-02-13
-----------

Directly parse .txt html file
instance: b
finished get html
finished get text
finished refine html
finished get html
finished get text
finished refine html
http://makeresearchgreatagain.com/sec_files/WESCO_International_Inc_2011-02-25_report.html
619
WESCO International Inc
2010-02-26
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/0000929008/000095012310018312/FilingSummary.xml
https://www.sec.gov/Archives/edgar/data/0000929008/0000950123-10-018312-index.htm
https://www.sec.gov/Archives/edgar/data/0000929008/0000950123-10-018312.txt
Directly parse .txt html file
instance: b
finished get html
finished get text
finished refine html
http://makeresearchgreatagain.com/sec_files/WESCO_International_Inc_2010-02-26_report.html
620
WESCO International Inc
2009-02-27
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/da

In [42]:
end = time()

In [43]:
print('Total runtime is ', str(end - start), ' seconds') 

Total runtime is  2248.371034860611  seconds


In [44]:
np.unique(firms_info['Ticker'])

array(['AECOM Technology Corp', 'Airgas Inc', 'Allergan Inc',
       'Allergan PLC', 'Bed Bath & Beyond Inc',
       'BioMarin Pharmaceutical Inc', 'Bottomline Technologies Inc',
       'Carver Corp', 'Comcast Corp', 'DaVita HealthCare Partners Inc',
       'DaVita Inc', 'Delek Us Holdings Inc', 'Dun & Bradstreet Corp',
       'Emerson Electric Co', 'Federal Signal Corp', 'Ferro Corp',
       'Hanger Inc', 'Hanger Orthopedic Group Inc',
       'Harman Intl Industries Inc', 'Hercules Offshore Inc',
       'Latteno Food Corp', 'Measurement Specialties Inc',
       'Microsoft Corp', 'Mistras Group Inc',
       'Monotype Imaging Holdings Inc', 'NCI Inc', 'NuStar Energy LP',
       'Omnicare Inc', 'Regency Energy Partners LP', 'ResMed Inc',
       'Superior Well Services Inc', 'Time Warner Cable Inc',
       'VCA Antech Inc', 'VCA Inc', 'Vonage Holdings Corp',
       'WESCO International Inc', 'Zayo Group Holdings Inc'], dtype=object)

In [45]:
print('Unable to process', failed_count, '10-Ks in total.')

Unable to process 1 10-Ks in total.


In [46]:
failed_tables

,Compnay Name,10-K Report Date,10-K Report Page URL,10-K Report .html URL
0,Omnicare Inc,2011-02-24,https://www.sec.gov/Archives/edgar/data/000035...,https://www.sec.gov/Archives/edgar/data/000035...


In [48]:
relevant_table_links

,Compnay Name,CIK,Notes Link,10-K Report Page URL,End Date of Fiscal Year
0,AECOM Technology Corp,0000868857,https://www.sec.gov/Archives/edgar/data/000086...,https://www.sec.gov/Archives/edgar/data/000086...,2017-09-29
1,AECOM Technology Corp,0000868857,https://www.sec.gov/Archives/edgar/data/000086...,https://www.sec.gov/Archives/edgar/data/000086...,2016-09-30
2,AECOM Technology Corp,0000868857,https://www.sec.gov/Archives/edgar/data/000086...,https://www.sec.gov/Archives/edgar/data/000086...,2015-10-02
3,AECOM Technology Corp,0000868857,https://www.sec.gov/Archives/edgar/data/000086...,https://www.sec.gov/Archives/edgar/data/000086...,2014-09-30
4,AECOM Technology Corp,0000868857,https://www.sec.gov/Archives/edgar/data/000086...,https://www.sec.gov/Archives/edgar/data/000086...,2013-09-30
...,...,...,...,...,...
332,WESCO International Inc,0000929008,http://makeresearchgreatagain.com/sec_files/WE...,https://www.sec.gov/Archives/edgar/data/000092...,2007-12-31
333,WESCO International Inc,0000929008,http://makeresearchgreatagain.com/sec_files/WE...,https://www.sec.gov/Archives/edgar/data/000092...,2006-12-31
334,Zayo Group Holdings Inc,0001608249,https://www.sec.gov/Archives/edgar/data/000160...,https://www.sec.gov/Archives/edgar/data/000160...,2017-06-30
335,Zayo Group Holdings Inc,0001608249,https://www.sec.gov/Archives/edgar/data/000160...,https://www.sec.gov/Archives/edgar/data/000160...,2016-06-30


In [49]:
relevant_table_links.iloc[7, 3]

'https://www.sec.gov/Archives/edgar/data/0000868857/0001047469-10-009946-index.htm'

In [50]:
# save file for RA
relevant_table_links.to_csv('reports_links_for_NonTop_acq.csv')
failed_tables.to_csv('failed_to_parse_for_NonTop_acq.csv')

In [ ]:
note_words = '''

On October 17, 2014, the Company completed the acquisition of the U.S. headquartered URS Corporation (URS), an international provider of engineering, construction, and technical services, by purchasing 100% of the outstanding shares of URS common stock. The purpose of the acquisition was to further diversify the Company's market presence and accelerate the Company's strategy to create an integrated delivery platform for customers. The Company paid total consideration of approximately $2.3 billion in cash and issued approximately $1.6 billion of AECOM common stock to the former stockholders and certain equity award holders of URS. In connection with the acquisition, the Company also assumed URS's senior notes totaling $1.0 billion, and upon the occurrence of a change in control of URS, the URS senior noteholders had the right to redeem their notes at a cash price equal to 101% of the principal amount of the notes. Accordingly, on October 24, 2014, the Company purchased $0.6 billion of URS's senior notes from the noteholders. See also Note 9, Debt. Additionally, the Company repaid in full URS's $0.6 billion 2011 term loan and $0.1 billion of URS's revolving line of credit. Net assets acquired
 
$

5,120.6 Acquisitions occurring in fiscal 2011 and 2010 were recorded using the acquisition method of accounting in accordance with the revised guidance on business combinations, which the Company adopted on April 1, 2009. The results of acquired companies' operations have been included in the Company's consolidated financial statements since the effective date of each respective acquisition.
Fiscal 2011 
During fiscal 2011, the Company purchased eight businesses. The largest of these businesses was Conley Gas, Ltd. ("Conley"), a supplier of pure gases to the specialty gas industry with historical annual sales of approximately $9 million. A total of $21.2 million in cash was paid for the eight businesses and for the settlement of holdback liabilities and contingent consideration arrangements associated with acquisitions that were completed in years prior to fiscal 2010. Transaction and other integration costs incurred in fiscal 2011 and included in the Company's fiscal 2011 results were insignificant. These businesses had aggregate historical annual sales of approximately $21 million. Net sales from current year acquisitions that were included in the Company's fiscal 2011 results were $7 million. The Company acquired these businesses in order to expand its geographic coverage and strengthen its national network of branch-store locations. 
Total cash consideration The fair value of trade receivables acquired with fiscal 2011 acquisitions was $2.3 million, which approximated gross contractual amounts receivable. Goodwill associated with fiscal 2011 acquisitions was $7.3 million and is deductible for income tax purposes. Goodwill largely consists of expected synergies resulting from the acquisitions, including the expansion of geographical coverage that will facilitate the sale of industrial, medical and specialty gases and related supplies. Intangible assets related to fiscal 2011 acquisitions represent customer relationships and non-competition agreements and amounted to $5.6 million and $1.9 million, respectively. See Note 7 for further information on goodwill and intangible assets, including the allocation by segment. 
In connection with acquisitions prior to the April 1, 2009 adoption of the revised guidance on business combinations, the Company is a party to contingent payment agreements that provide for additional consideration to be paid to sellers if the future earnings of the acquired businesses exceed predetermined amounts. Amounts payable under contingent payment agreements continue through 2019 and are limited to $7.9 million. Such amounts, if paid, will be treated as additional costs of the acquisitions. 
Note 2:  Acquisitions and Collaborations
LiRIS Acquisition
On August 13, 2014, the Company completed the acquisition of LiRIS Biomedical, Inc. (LiRIS), a clinical-stage specialty pharmaceutical company based in the United States focused on developing a pipeline of innovative treatments for bladder diseases, for an upfront payment of $67.5 million, plus up to an aggregate of $295.0 million in payments contingent upon achieving certain future development milestones and up to an aggregate of $225.0 million in payments contingent upon achieving certain commercial milestones. The estimated fair value of the contingent consideration as of the acquisition date was $170.5 million. The acquisition was funded from current cash and equivalents balances.
The Company recognized tangible and intangible assets acquired and liabilities assumed in connection with the LiRIS acquisition based on their estimated fair values at the acquisition date. The excess of the purchase price over the fair value of net assets acquired was recognized as goodwill. The goodwill acquired in the LiRIS acquisition is not deductible for federal income tax purposes. In connection with the acquisition, the Company acquired assets with a fair value of $307.8 million, consisting of intangible assets of $238.0 million and goodwill of $69.8 million, and assumed non-current deferred tax liabilities of $69.8 million. As of December 31, 2014, the total estimated fair value of the contingent consideration of $164.8 million was included in “Other liabilities.” 
The intangible assets consist of an in-process research and development asset of $225.3 million and a patented device technology asset of $12.7 million associated with LiRIS' proprietary delivery system that has an estimated useful life of 16 years. The in-process research and development asset relates to LiRIS’ lead investigational product for the treatment of interstitial cystitis and bladder pain syndrome which is currently in Phase II clinical trials. The estimated fair value of the in-process research and development asset was determined based on the use of a discounted cash flow model using an income approach. The in-process research and development asset is classified as an indefinite-lived intangible asset until the successful completion and commercialization or abandonment of the associated research and development efforts.
Goodwill represents the excess of the LiRIS purchase price over the sum of the fair values assigned to assets acquired less liabilities assumed. The LiRIS acquisition has the potential to broaden the Company's product offering in urology in a disease state with high unmet needs, which the Company believes supports the amount of goodwill recognized as a result of the purchase price paid for LiRIS, in relation to other acquired tangible and intangible assets.
The Company estimated the fair value of the contingent consideration liabilities related to the achievement of future development and regulatory milestones by assigning an achievement probability to each potential milestone and discounting the associated cash payment to its present value using a risk-adjusted rate of return. The Company estimated the fair value of the contingent consideration liabilities associated with sales milestones by employing Monte Carlo simulations to estimate the volatility and systematic relative risk of acquired product revenues and discounting the associated cash payment amounts to their present values using a credit-risk-adjusted interest rate.
MAP Acquisition
On March 1, 2013, the Company completed the acquisition of MAP Pharmaceuticals, Inc. (MAP), a biopharmaceutical company based in the United States focused on developing and commercializing new therapies in neurology, including SempranaTM, formerly referred to as Levadex®, an orally inhaled drug for the potential acute treatment of migraine in adults, for an aggregate purchase price of approximately $871.7 million, net of cash acquired. The acquisition was funded from a combination of current cash and equivalents and short-term investments.
The Company recognized tangible and intangible assets acquired and liabilities assumed in connection with the MAP acquisition based on their estimated fair values at the acquisition date. The excess of the purchase price over the fair value of net assets acquired was recognized as goodwill. The goodwill acquired in the MAP acquisition is not deductible for federal income tax purposes. In connection with the acquisition, the Company acquired assets with a fair value of $1,233.6 million, consisting of current assets of $2.3 million, property, plant and equipment of $7.7 million, other non-current assets of $0.3 million, deferred tax assets of $132.7 million, intangible assets of $915.6 million and goodwill of $175.0 million, and assumed liabilities of $361.9 million, consisting of current liabilities of $27.3 million and deferred tax liabilities of $334.6 million.
The intangible assets consist of an in-process research and development asset of $683.5 million associated with SempranaTM and a patented device technology asset of $232.1 million associated with MAP's proprietary Tempo® delivery system that has an estimated useful life of 15 years.
Goodwill represents the excess of the MAP purchase price over the sum of the amounts assigned to assets acquired less liabilities assumed. The MAP acquisition broadens the Company's product offering for the treatment of migraine headaches and MAP's proprietary drug particle and inhalation technology provides the potential for new product development opportunities, which the Company believes support the amount of goodwill recognized as a result of the purchase price paid for MAP, in relation to other acquired tangible and intangible assets.
Exemplar Acquisition
On April 12, 2013, the Company completed the acquisition of Exemplar Pharma, LLC (Exemplar), a third party contract manufacturer for MAP's Tempo® delivery system, for an aggregate purchase price of approximately $16.1 million, net of cash acquired. Prior to the acquisition, the Company also had a $1.9 million payable to Exemplar, which was effectively settled upon the acquisition. In connection with the acquisition, the Company acquired assets with a fair value of $16.6 million, consisting of current assets of $0.5 million, property, plant and equipment of $2.1 million and goodwill of $14.0 million, and assumed current liabilities of $0.5 million. The goodwill acquired in the Exemplar acquisition is deductible for federal income tax purposes.
Medytox Collaboration
On September 25, 2013, the Company announced that it had entered into a license agreement with Medytox, Inc. (Medytox), contingent on obtaining certain government approvals. In January 2014, the Company closed the transaction. Under the terms of the agreement, the Company made an upfront payment to Medytox of $65.0 million in January 2014 and Medytox granted the Company exclusive rights, worldwide outside of Korea with co-exclusive rights in Japan, to develop and, if approved, commercialize certain neurotoxin product candidates currently in development, including a potential liquid-injectable product. The upfront payment of $65.0 million was recorded as research and development (R&D) expense in the first quarter of 2014 because the technology has not yet achieved regulatory approval. The terms of the agreement also include potential future development milestone payments of up to $116.5 million and potential future sales milestone payments of up to $180.5 million, as well as potential future royalty payments. In the third quarter of 2014, the Company made a development milestone payment to Medytox of $15.0 million, which was recorded as R&D expense because the technology has not yet achieved regulatory approval.
Other Acquisitions and Collaborations
In March 2014, the Company completed the acquisition of certain assets from Aline Aesthetics, LLC and Tautona Group, L.P. for an upfront payment of $10.0 million and potential future payments for certain milestone events. The Company accounted for the acquisition as a purchase of net assets. The acquired assets primarily consist of intellectual property related to technology under development for use as a dermal filler that has not achieved regulatory approval. The upfront payment was accrued and recorded as R&D expense in the first quarter of 2014 and was paid in the second quarter of 2014.
In November 2013, the Company purchased a noncontrolling interest in a subsidiary from a minority shareholder for $18.0 million. The Company accounted for the purchase as an equity transaction and recorded the difference between the cash consideration and the carrying amount of the noncontrolling interest, including its share of accumulated other comprehensive income, as a decrease in additional paid-in capital of $1.3 million. 
On September 10, 2013, the Company entered into a license and collaboration agreement with a third party pursuant to which the Company obtained exclusive global rights to research, manufacture and commercialize certain technologies for the treatment of ocular disease. Under the terms of the agreement, the Company made a $6.5 million upfront payment, which was recorded as R&D expense in the third quarter of 2013 because the technology has not yet achieved regulatory approval. The terms of the agreement also include potential future payments to the third party related to the Company’s achievement of development, regulatory and sales milestone events, as well as potential future royalty payments.
In connection with various business development transactions where the Company has outlicensed its technology to third parties, the Company has aggregate potential future milestone receipts of approximately $45.9 million as of December 31, 2014, none of which are individually significant. Of that amount, approximately $3.5 million relates to achievement of certain development milestones, approximately $17.0 million relates to achievement of certain regulatory milestones, and approximately $25.4 million relates to achievement of certain commercial sales milestones. Due to the challenges associated with developing and obtaining approval for pharmaceutical products, there is substantial uncertainty whether any of the future milestones will be achieved. The Company evaluates whether milestone payments are substantive based on the facts and circumstances associated with each milestone payment.
The Company believes that the fair values assigned to the assets acquired and liabilities assumed for its acquisitions were based on reasonable assumptions. The Company's fair value estimates may change during the allowable measurement period, which is up to one year from the acquisition date, if additional information becomes available.
NOTE 4 — Acquisitions and Other Agreements
Acquisition of Warner Chilcott
On October 1, 2013, the Company completed the Warner Chilcott Acquisition in a stock for stock transaction for a value, including the assumption of debt, of $9.2 billion. Warner Chilcott was a leading specialty pharmaceutical company focused on the women’s healthcare, gastroenterology, urology and dermatology segments of the branded pharmaceuticals market, primarily in North America.
Recognition and Measurement of Assets Acquired and Liabilities Assumed at Fair Value
The transaction has been accounted for using the acquisition method of accounting. This method requires that assets acquired and liabilities assumed in a business combination be recognized at their fair values as of the acquisition date. As of December 31, 2013, certain amounts relating to SRA reserves have not been finalized. The finalization of these matters may result in changes to goodwill and the Company expects to finalize such matters in 2014.
Consideration
The total consideration for the Warner Chilcott Acquisition of $5,833.9 million is comprised of the equity value of shares that were outstanding and vested prior to October 1, 2013 ($5,761.3 million) and the portion of outstanding equity awards deemed to have been earned as of October 1, 2013 ($72.6 million). The portion deemed not to have been earned ($77.4 million) as of October 1, 2013 will be expensed over the remaining future vesting period, including $45.4 million relating to Warner Chilcott restructuring charges recognized in the year ended December 31, 2013.
Inventories
The fair value of inventories acquired included a step-up in the value of inventories of $408.3 million. In the year ended December 31, 2013, the Company recognized $173.5 million as a component of cost of sales as the inventory acquired on October 1, 2013 was sold to the Company’s customers.
IPR&D and Intangible Assets
IPR&D intangible assets represent the value assigned to acquired R&D projects that, as of the acquisition date, had not established technological feasibility and had no alternative future use. The IPR&D intangible assets are capitalized and accounted for as indefinite-lived intangible assets and will be subject to impairment testing until completion or abandonment of the projects. Upon successful completion of each project and launch of the product, the Company will make a separate determination of the estimated useful life of the IPR&D intangible asset and the related amortization will be recorded as an expense over the estimated useful life (“IPR&D Acquisition Accounting”). Intangible assets represent CMPs and IPR&D and have an estimated weighted average useful life of 2.7 years.
The estimated fair value of the IPR&D and identifiable intangible assets was determined using the “income approach,” which is a valuation technique that provides an estimate of the fair value of an asset based on market participant expectations of the cash flows an asset would generate over its remaining useful life. Some of the more significant assumptions inherent in the development of those asset valuations include the estimated net cash flows for each year for each asset or product (including net revenues, cost of sales, R&D costs, selling and marketing costs and working capital/asset contributory asset charges), the appropriate discount rate to select in order to measure the risk inherent in each future cash flow stream, the assessment of each asset’s life cycle, the potential regulatory and commercial success risks, competitive trends impacting the asset and each cash flow stream as well as other factors (the “IPR&D and Intangible Asset Valuation Technique”). The discount rates used to arrive at the present value at the acquisition date of CMPs was 8.0% and for IPR&D ranged from 8.0% to 9.0%, to reflect the internal rate of return and incremental commercial uncertainty in the cash flow projections. No assurances can be given that the underlying assumptions used to prepare the discounted cash flow analysis will not change. For these and other reasons, actual results may vary significantly from estimated results.
 
The following table identifies the summarized amounts recognized and the weighted average useful lives of intangible assets:

Acquisition of Warner Chilcott
On October 1, 2013, the Company completed the Warner Chilcott Acquisition in a stock for stock transaction for a value, including the assumption of debt, of $9.2 billion. Warner Chilcott was a leading specialty pharmaceutical company focused on the women’s healthcare, gastroenterology, urology and dermatology segments of the branded pharmaceuticals market, primarily in North America.
Recognition and Measurement of Assets Acquired and Liabilities Assumed at Fair Value
The transaction has been accounted for using the acquisition method of accounting. This method requires that assets acquired and liabilities assumed in a business combination be recognized at their fair values as of the acquisition date. As of December 31, 2013, certain amounts relating to SRA reserves have not been finalized. The finalization of these matters may result in changes to goodwill and the Company expects to finalize such matters in 2014

Acquisition of Medicines360
On June 11, 2013, the Company entered into an exclusive license agreement with Medicines360 to market, sell and distribute Medicines360’s LNG20 intrauterine device (“LNG 20”) in the U.S. and in Canada for a payment of approximately $52.3 million. According to the terms of the agreement, the Company is also required to pay Medicines360 certain regulatory and sales based milestone payments totaling up to $125.0 million plus royalties (the “Medicines360 Acquisition”). Medicines360 retained the rights to market the product in the U.S. public sector, including family planning clinics that provide services to low-income women. LNG20, originally developed by Uteron Pharma S.P.R.L. in Belgium (now a subsidiary of the Company), is designed to deliver 20 mcg of levonorgestrel per day for the indication of long-term contraception, and is currently in Phase III clinical trials in the United States. Pending FDA approval, the LNG20 product could be launched in the U.S. as early as 2014. The transaction has been accounted for using the acquisition method of accounting. This method requires that assets acquired and liabilities assumed in a business combination be recognized at their respective fair values as of the acquisition date. In connection with the acquisition, the Company recorded $191.7 million in IPR&D, $6.7 million in prepaid R&D and contingent consideration of $146.1 million

Unaudited Pro Forma Results of Operations
Pro forma results of operations have not been presented because the effect of the Medicines360 Acquisition was not material.
 Acquisition of Actavis Group
On October 31, 2012, the Company completed the Actavis Group Acquisition. The Actavis Group was a privately held generic pharmaceutical company specializing in the development, manufacture and sale of generic pharmaceuticals. With the Actavis Group Acquisition, the Company significantly expanded its international market presence in established markets including Europe and MEAAP. In addition, the acquisition expanded the Company’s product portfolio and pipeline in modified release, solid oral dosage and transdermal products into semi-solids, liquids and injectables. Actavis Group results are included in the Actavis Pharma and Actavis Specialty Brands segments as of the acquisition date.
The Company funded the cash portion of the transaction through a combination of term loan borrowings and senior unsecured notes. For additional information, refer to “Note 13 — Long-term Debt.”
Recognition and Measurement of Assets Acquired and Liabilities Assumed at Fair Value
The transaction has been accounted for using the acquisition method of accounting. The following table summarizes the final fair values of the tangible and identifiable intangible assets acquired and liabilities assumed at the acquisition date:
Acquisition of Ascent Pharmahealth Ltd.
On January 24, 2012, the Company acquired all of the outstanding equity of Ascent Pharmahealth Ltd. (“Ascent”) the Australian and Southeast Asian generic pharmaceutical business of Strides Arcolab Ltd. for AU$376.6 million, or approximately $392.6 million, including working capital adjustments (the “Ascent Acquisition”). As a result of the acquisition, the Company enhanced its commercial presence in Australia and gained selling and marketing capabilities in Southeast Asia. In Australia, Ascent markets generic, brands, OTC and dermatology and skin care products. In Southeast Asia, Ascent markets generic and OTC products. Ascent’s Southeast Asian business includes commercial operations in Singapore, Malaysia, Hong Kong, Vietnam and Thailand. Ascent operates a manufacturing facility in Singapore for generic products in Southeast Asian markets. Ascent’s results are included in the Actavis Pharma segment as of the acquisition date.
Pro forma results of operations have not been presented because the effect of the acquisition was not material.
Acquisition of Specifar
On May 25, 2011, the Company and each of the shareholders (together, the “Sellers”) of Paomar PLC (“Paomar”) entered into a stock purchase agreement pursuant to which the Company purchased all of the outstanding equity of Paomar for cash totaling €400.0 million, or approximately $561.7 million at closing, subject to a net of working capital adjustment of €1.5 million, or approximately $2.2 million, and certain contingent consideration (the “Specifar Acquisition”). Paomar is a company incorporated under the laws of Cyprus and owner of 100 percent of the shares of Specifar Commercial Industrial Pharmaceutical, Chemical and Construction Exploitations Societe Anonyme, a company organized under the laws of Greece. Specifar owns 100 percent of the shares of Alet Pharmaceuticals Industrial and Commercial Societe Anonyme (“Alet”). The contingent consideration due to the Specifar Acquisition (not to exceed an aggregate total of €40.0 million) is based on the gross profits on sales of the generic tablet version of Nexium® (esomeprazole) developed by Specifar during its first five years of sales in countries including major markets in Europe, Asia and Latin America, as well as in Canada. For additional information on the contingent payment, refer to “NOTE 20 — Fair Value Measurements”.
Through the Specifar Acquisition, the Company gained a generic pharmaceuticals product development company that develops and out-licenses generic pharmaceutical products primarily in Europe. In addition, the acquisition enhanced the Company’s commercial presence in key European markets by providing a portfolio of products and provides a commercial presence in the branded-generic Greek pharmaceuticals market, including the Specifar and Alet brands of products. The Company funded the transaction using cash on hand and borrowings from the Company’s credit facility. Specifar results are included in the Actavis Pharma segment subsequent to the acquisition date.

Property Acquisitions 

Properties Under Development
During the year ended December 31, 2016, the Company secured the following in-process development properties for approximately $39.6 million. Total cash consideration of $40.7 million consisted of escrow deposits and cash paid at closing:

Since their respective acquisition dates, the operating properties discussed above contributed a combined $1.9 million of revenues for the year ended December 31, 2016.  These properties had a combined net loss of $0.1 million for the year ended December 31, 2016, which includes $0.3 million of acquisition-related costs such as broker fees, due diligence costs and legal and accounting fees that are included in wholly-owned properties operating expense on the accompanying consolidated statements of comprehensive income. 

During 2015, the Company acquired eight wholly-owned properties containing 4,061 beds for a combined purchase price of approximately $378.3 million. As part of these transactions, the Company assumed approximately $69.4 million of mortgage debt and issued 343,895 Common OP Units, valued at $41.24 per unit. 

During 2014, the Company acquired one property comprised of 610 beds located near the University of Georgia and a property containing a hotel near the University of Colorado campus. The existing hotel at the University of Colorado was demolished in the fourth quarter 2014 and a new 398-bed student housing facility was constructed and opened for occupancy in August 2016. The total consideration for these two acquisitions was approximately $75.1 million.



'''

In [ ]:
import string
string.punctuation

In [ ]:
import re
words_list = re.sub(r'[^\w\s]','',note_words).replace('\n', '').lower().split()
words_list

In [ ]:
unique_elements, counts_elements = np.unique(words_list, return_counts=True)
sort_index = np.argsort(counts_elements)[::-1]
unique_elements = unique_elements[sort_index]
counts_elements = counts_elements[sort_index]
unique_elements[1:100]

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#convert list to string and generate
unique_string=(' ').join(words_list)
wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
#plt.savefig("wordcloud_in_acq_notes"+".png", bbox_inches='tight')
plt.show()
plt.close()

#i guess most frequent are:
#['acquisition', 'acquire', 'merger', 'business combination', 'asset', 'million', 'billion',
#'fair', 'value', 'purchase price', 'price', 'paid', 'goodwill']

In [ ]:
#circle instead of rectangle
x, y = np.ogrid[:300, :300]

mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)


wc = WordCloud(background_color="white", repeat=True, mask=mask).generate(' '.join(words_list))

plt.axis("off")
plt.savefig("wordcloud_in_acq_notes_circle"+".png", bbox_inches='tight')
plt.imshow(wc, interpolation="bilinear")
plt.show()
plt.close()

In [ ]:
#Here is for changing the fileppath for files I generated

In [ ]:
relative_fp = os.path.realpath(os.getcwd())

reports_fp = relative_fp + '\\reports_links_for_NonTop_acq.csv'
reports_df = pd.read_csv(reports_fp)
reports_df.drop('Unnamed: 0', axis=1)

for i in range(len(reports_df)):
    link_col_index = 4
    if reports_df.iloc[i, 4].endswith('.html'):
        reports_df.iloc[i, 4] = r'http://makeresearchgreatagain.com/sec_files/' + reports_df.iloc[i, 4]    

In [ ]:
reports_df

In [ ]:
# save file
reports_df.to_csv('reports_links_for_NonTop_acq.csv')